In [1]:
%%capture

import warnings
warnings.filterwarnings('ignore')

import calitp_data_analysis.magics

import altair as alt
import pandas as pd
from calitp_data_analysis import calitp_color_palette as cp
from IPython.display import HTML, Markdown, display
from new_transit_metrics_utils import (
    GCS_FILE_PATH,
    make_line,
    make_long,
    make_scatter,
    sum_by_group,
)
from scipy.stats import zscore

# alt.renderers.enable("html")
alt.data_transformers.enable("default", max_rows=None)

WIDTH = 300
HEIGHT = 150

In [2]:
rtpa = "Sacramento Area Council of Governments"

In [3]:
%%capture_parameters
rtpa

# {rtpa}
## New Transit Performance Metrics

UCLA Institute of Transportation Studies (UCLA ITS) suggests
>Updating the policy and legislation that governs state transit funding could help make expenditures more effective and better aligned with the state’s goals of VMT and GHG reduction, which transit can achieve only through increased ridership.

Some of the measures UCLA ITS recommends matching performance metrics to transit market context and suggest a contemporary view of transit operating conditions. Namely, transit-oriented vs. auto-oriented areas, which can be measured using cost-efficiency and service-effectiveness metrics.

The charts below display the cost-efficiency metrics (opex_per_vrm/vrh/upt) and service-effectiveness metrics (upt_per_vrm/vrh) by different categories.

In [4]:
# read in data
df = pd.read_parquet(
    f"{GCS_FILE_PATH}raw_transit_performance_metrics_data.parquet",
    filters=[("RTPA", "==", rtpa)],
)

In [5]:
# variable lists
agency_cols = ["ntd_id", "agency_name", "RTPA"]
mode_cols = ["mode", "RTPA"]
tos_cols = ["service", "RTPA"]
val_cols = [
    "opex_per_vrh",
    "opex_per_vrm",
    "upt_per_vrh",
    "upt_per_vrm",
    "opex_per_upt",
]

cost_eff = [
    "opex_per_vrh",
    "opex_per_vrm",
    "opex_per_upt",
]  # just cost-effecieny metrics
service_eff = ["upt_per_vrh", "upt_per_vrm"]  # just service effectiveness metrics

In [6]:
by_agency = sum_by_group(df, agency_cols)
by_mode = sum_by_group(df, mode_cols)
by_tos = sum_by_group(df, tos_cols)

In [7]:
by_agency_long = make_long(df=by_agency, group_cols=agency_cols, value_cols=val_cols)
by_mode_long = make_long(df=by_mode, group_cols=mode_cols, value_cols=val_cols)
by_tos_long = make_long(df=by_tos, group_cols=tos_cols, value_cols=val_cols)

## New Performance Metrics

| Metric type          | Metric example                  | Implicit Goal(s)                       | Advantages                                   | Limitations                                  |
|----------------------|---------------------------------|---------------------------------------|----------------------------------------------|----------------------------------------------|
| Cost-efficiency     | Operating cost per revenue hour (opex_per_vrh) | Reduce costs                         | Useful in both financial and service planning | Favors high labor productivity in dense, congested areas; does not track use |
|                      | Operating cost per revenue mile (opex_per_vrm) |                                       |                                              |                                              |
|                      | Operating cost per vehicle trip (opex_per_upt) |                                       |                                              |                                              |
| Service-effectiveness| Passengers per revenue-vehicle hour (upt_per_vrh) | Increase ridership; reduce poorly patronized service | Useful for service planning; emphasizes what matters to riders | Favors high ridership; does not track costs   |
|                      | Passengers per revenue-vehicle mile (upt_per_vrm) | Increase ridership; reduce low-ridership route miles/segments | Useful for service planning                | Favors high ridership and fast vehicle speeds; does not track costs |


### By Agencies

In [8]:
# all agencies
make_line(
    by_agency_long,
    x_col="year:T",
    y_col="value",
    facet="variable:N",
    color="agency_name:N",
    chart_title="New performance Metrics per Transit Agency",
    ind_axis=True,
)

alt.FacetChart(...)

### By Mode

In [9]:
make_line(
    by_mode_long,
    x_col="year:T",
    y_col="value",
    facet="variable:N",
    color="mode:N",
    chart_title="New Performance Metrics per Mode",
    ind_axis=True,
)

alt.FacetChart(...)

### By Type of Service

In [10]:
make_line(
    by_tos_long,
    x_col="year:T",
    y_col="value",
    facet="variable:N",
    color="service:N",
    chart_title="New Performance Metrics per Type of Service",
    ind_axis=True,
)

alt.FacetChart(...)

## Cost-efficiency metrics
Cost-efficiency measures inputs to outputs: For example, the cost of operating an hour of transit service.

Per the UCLA ITS Paper
>Transit-oriented markets (which are predominantly urban), transit service tends to be relatively service-effective. But high operating costs on these (mostly) older, larger systems can inhibit efforts to improve ridership by adding service. In such contexts, assessing systems with an emphasis on **cost-efficiency (i.e., the cost of operating an hour of service)** grounds would provide incentives for agencies to **manage their costs** so as to be able to provide more service with available funding.

### Operating cost per VRH
Lower is better

In [11]:
make_scatter(
    data=df[df["year"] == "2023"],
    y_ax="opexp_total",
    x_ax="vrh",
    color="reporter_type",
    log_scale=True,
    lin_y_ax=True,
    chart_title=""
).facet(
    facet=alt.Facet("mode", 
                    title="Operating Cost per VRH, by mode"
                   ),
    columns=2,
)

0 rows with zero or negative values excluded due to log scale.


alt.FacetChart(...)

In [12]:
make_scatter(
    data=df[df["year"] == "2023"],
    y_ax="opexp_total",
    x_ax="vrh",
    color="reporter_type",
    log_scale=True,
    lin_y_ax=True,
    chart_title=""
).facet(
    facet=alt.Facet("service", 
                    title="Operating Cost per VRH, by Type of Service"
                   ),
    columns=2,
)

0 rows with zero or negative values excluded due to log scale.


alt.FacetChart(...)

### Operating cost per VRM
Lower is better

In [13]:
make_scatter(
    data=df[df["year"] == "2023"],
    y_ax="opexp_total",
    x_ax="vrm",
    color="reporter_type",
    log_scale=True,
    lin_y_ax=True,
    chart_title=""
).facet(
    facet=alt.Facet("mode",title="Operating Cost per VRH, by mode"),
    columns=2,
)

0 rows with zero or negative values excluded due to log scale.


alt.FacetChart(...)

In [14]:
make_scatter(
    data=df[df["year"] == "2023"],
    y_ax="opexp_total",
    x_ax="vrm",
    color="reporter_type",
    log_scale=True,
    lin_y_ax=True,
    chart_title=""
).facet(
    facet=alt.Facet("service",title="Operating Cost per VRH, by Type of Service"),
    columns=2,
)

0 rows with zero or negative values excluded due to log scale.


alt.FacetChart(...)

### Operating cost per trip
Lower is better

In [15]:
make_scatter(
    data=df[df["year"] == "2023"],
    y_ax="opexp_total",
    x_ax="upt",
    color="reporter_type",
    log_scale=True,
    lin_y_ax=True,
    chart_title=""
).facet(
    facet=alt.Facet("mode",
                   title="Operating Cost per UPT, by mode"),
    columns=2,
)

0 rows with zero or negative values excluded due to log scale.


alt.FacetChart(...)

In [16]:
make_scatter(
    data=df[df["year"] == "2023"],
    y_ax="opexp_total",
    x_ax="upt",
    color="reporter_type",
    log_scale=True,
    lin_y_ax=True,
    chart_title=""
).facet(
    facet=alt.Facet("service",
                   title="Operating Cost per UPT, by Type of Service"),
    columns=2,
)

0 rows with zero or negative values excluded due to log scale.


alt.FacetChart(...)

## Service-effectiveness metrics
Service-effectiveness measures outputs to consumption: For example, passenger boardings per service hour.

Per the UCLA ITS Paper
>[In] more auto-oriented markets, transit operators tend to be relatively cost-efficient, in that they have lower operating costs but serve fewer riders. In this context, assessing systems with an emphasis on **service-effectiveness (i.e., passenger boardings per service hour)** will motivate operators to **improve ridership** by changing service hours, routes, and fares to better match local demand. Agencies might also implement fare programs with schools and other institutions, and even work with municipalities on improving land use around transit in order to increase the relative attractiveness of transit service.

### Passengers per VRH
Higher is better

In [17]:
make_scatter(
    data=df[df["year"] == "2023"],
    y_ax="upt",
    x_ax="vrh",
    color="reporter_type",
    log_scale=True,
    lin_y_ax=True,
    chart_title=""
).facet(
    facet=alt.Facet("mode",
                   title="Passengers per VRH, by mode"),
    columns=2,
)

0 rows with zero or negative values excluded due to log scale.


alt.FacetChart(...)

In [18]:
make_scatter(
    data=df[df["year"] == "2023"],
    y_ax="upt",
    x_ax="vrh",
    color="reporter_type",
    log_scale=True,
    lin_y_ax=True,
    chart_title=""
).facet(
    facet=alt.Facet("service",
                   title="Passengers per VRH, by Type of Service"),
    columns=2,
)

0 rows with zero or negative values excluded due to log scale.


alt.FacetChart(...)

### Passengers per VRM
Higher is better

In [19]:
make_scatter(
    data=df[df["year"] == "2023"],
    y_ax="upt",
    x_ax="vrm",
    color="reporter_type",
    log_scale=True,
    lin_y_ax=True,
    chart_title= ""
).facet(
    facet=alt.Facet("mode",title="Passengers per VRM, by mode"),
    columns=2,
)

0 rows with zero or negative values excluded due to log scale.


alt.FacetChart(...)

In [20]:
make_scatter(
    data=df[df["year"] == "2023"],
    y_ax="upt",
    x_ax="vrm",
    color="reporter_type",
    log_scale=True,
    lin_y_ax=True,
    chart_title=""
).facet(
    facet=alt.Facet("service",title="Passengers per VRM, by Type of Service"),
    columns=2,
)

0 rows with zero or negative values excluded due to log scale.


alt.FacetChart(...)